## Time Series EDA
- single continuous variable over (uniformly distributed) time
- identify trend and seasonality

### Setup
°F = (K - 273.15) x 1.8000 + 32.00

In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [2]:
# plotting defaults
plt.rc('figure', figsize=(13, 7))
plt.style.use('seaborn-whitegrid')
plt.rc('font', size=16)

In [3]:
# reading in data from csv, changing index to date column, sorting by date
df = pd.read_csv('sa_temps.csv', index_col='datetime', parse_dates=True).sort_index()

# converting temperature type to Fahrenheit
df.temp = (df.temp - 273.15) * 1.8 + 32

# printing date range and df shape
print('Date Range:', df.index.min(), 'to', df.index.max())
print('Shape:', df.shape)

# previewing data
df.head(10)

Date Range: 2012-10-01 13:00:00 to 2017-11-30 00:00:00
Shape: (45252, 1)


,temp
datetime,
2012-10-01 13:00:00,61.052000
2012-10-01 14:00:00,61.076568
2012-10-01 15:00:00,61.139294
2012-10-01 16:00:00,61.202020
2012-10-01 17:00:00,61.264747
2012-10-01 18:00:00,61.327473
2012-10-01 19:00:00,61.390200
2012-10-01 20:00:00,61.452926
2012-10-01 21:00:00,61.515652
